In [1]:
from dotenv import load_dotenv
load_dotenv()
import rag_helpers as rh


client = rh.get_weaviate_client()

weaviate client connected: True


In [3]:

embeddings = rh.get_embedding_model().get_text_embedding(
    "OpenAI new Embeddings models is great."
)

print(f"embeddings vector: {embeddings[:5]}.....")

print("\n")

print(f"embeddings vector length: {len(embeddings)}")

embeddings vector: [-0.007724063005298376, 0.039427097886800766, -0.017061689868569374, -0.02000819891691208, 0.024540210142731667].....


embeddings vector length: 3072


In [14]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser

TESLA_DOC = './.data/Tesla Cybertruck Owners Manual.md'
docs = SimpleDirectoryReader(input_files=[TESLA_DOC]).load_data()

parser = SimpleNodeParser()
nodes = parser.get_nodes_from_documents(docs)
print(f"Loaded {len(nodes):,} nodes from '{TESLA_DOC}'")

Loaded 1,281 nodes from './.data/Tesla Cybertruck Owners Manual.md'


In [16]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.weaviate import WeaviateVectorStore

vector_store = WeaviateVectorStore(weaviate_client = client, index_name=rh.COLLECTION_NAME)
storage_context = StorageContext.from_defaults(vector_store = vector_store)
index = VectorStoreIndex(nodes, embed_model=rh.get_embedding_model(), storage_context = storage_context)
print(f"setup index: {index.index_id} for {rh.COLLECTION_NAME}")

setup index: 82012b44-b6a2-4551-a926-d6bce1248b18 for TeslaCybertruckOwnersManual


In [17]:
query_engine = index.as_query_engine()
response = query_engine.query("How can I maximize my charge?")
print(response)

client.close()

To maximize your charge, you should follow these tips: slow down your driving and avoid rapid acceleration, use regenerative braking when slowing down, limit the use of resources like heating and air conditioning, precondition your vehicle using the mobile app, set a schedule for preconditioning and charging, ensure proper wheel alignment and tire pressure, install wheel covers to reduce wind resistance, remove unnecessary cargo to lighten the load, fully raise windows and close the tonneau cover, disable energy-consuming features when not needed, and keep the vehicle plugged in when idle to prevent excessive energy consumption.
